# Collection setup and data load

## Get keys and urls

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY[:20]}")
print(f"OpenAI URL: {OPENAI_URL}")


Weaviate Key:root-user-key
OpenAI API Key: sk-proj-iuwKF1Q94jnW
OpenAI URL: https://api.openai.com


## Connect to Weaviate

You need to pass in your Weaviate Cloud URL and KEY.

In [6]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

True


## Create a collection with a vectorizer

* [Weaviate Docs - collection creation and configuration](https://weaviate.io/developers/weaviate/manage-data/collections)
* [Weaviate integrated embedding models](https://weaviate.io/developers/weaviate/model-providers/weaviate/embeddings)

Examples of other embedding models:
* [Cohere](https://weaviate.io/developers/weaviate/model-providers/cohere/embeddings)
* [HuggingFace 🤗](https://weaviate.io/developers/weaviate/model-providers/huggingface/embeddings)
* [Ollama (self-hosted)](https://weaviate.io/developers/weaviate/model-providers/ollama/embeddings)
* [OpenAI](https://weaviate.io/developers/weaviate/model-providers/openai/embeddings)

In [7]:
from weaviate.classes.config import Configure

if client.collections.exists("Jeopardy"):
    client.collections.delete("Jeopardy")

# Create a collection - with Weaviate vectorizer
client.collections.create(
    name="Jeopardy",
    vector_config=Configure.Vectors.text2vec_openai(
        model="text-embedding-3-small",
        base_url=OPENAI_URL
    )

    # TODO: add text2vec_openai vectorizer (or your favorite vectorizer) - with:
    # * model - text-embedding-3-small (or your favorite model)
    # * base_url - use OPENAI_URL 
    
)

## Import data
### Sample Data

In [8]:
import json

with open("./jeopardy_tiny.json") as file:
    data_10 = json.load(file)

print(json.dumps(data_10[0:2], indent=2))

[
  {
    "Category": "SCIENCE",
    "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
    "Answer": "Liver"
  },
  {
    "Category": "ANIMALS",
    "Question": "It's the only living mammal in the order Proboseidea",
    "Answer": "Elephant"
  }
]


### Insert Many

> `insert_many` is only used for inserting small batches of data - must complete within the timeout.

[Weaviate Docs - insert many](https://weaviate.io/developers/weaviate/manage-data/import)

In [9]:
# Insert data
jeopardy = client.collections.use("Jeopardy")
jeopardy.data.insert_many(data_10)
# TODO: get Jeopardy collection
# TODO: insert data_10

BatchObjectReturn(_all_responses=[UUID('694ced5b-f355-449d-a866-0701561621d5'), UUID('d2542448-d164-4871-bb74-28c7a7f2468b'), UUID('d9cd6bdb-c1ed-431b-9c23-ef554c917d4d'), UUID('8500216d-0a11-4317-8cd7-a69a499ba0d6'), UUID('7aa8a88a-92e1-4a3c-9a3b-660fbfd3a507'), UUID('e78c2e7c-ec5b-409e-ac31-d8954fca5188'), UUID('74f3e196-f573-4759-80e4-a5ae0b712e51'), UUID('343cb976-b2e9-4e6f-add3-7a1c326ffa8d'), UUID('c0193d7c-53bd-4467-823f-613825dce729'), UUID('8fa3e0ab-00cc-46e9-adad-04848f00d7b4')], elapsed_seconds=0.7261512279510498, errors={}, uuids={0: UUID('694ced5b-f355-449d-a866-0701561621d5'), 1: UUID('d2542448-d164-4871-bb74-28c7a7f2468b'), 2: UUID('d9cd6bdb-c1ed-431b-9c23-ef554c917d4d'), 3: UUID('8500216d-0a11-4317-8cd7-a69a499ba0d6'), 4: UUID('7aa8a88a-92e1-4a3c-9a3b-660fbfd3a507'), 5: UUID('e78c2e7c-ec5b-409e-ac31-d8954fca5188'), 6: UUID('74f3e196-f573-4759-80e4-a5ae0b712e51'), 7: UUID('343cb976-b2e9-4e6f-add3-7a1c326ffa8d'), 8: UUID('c0193d7c-53bd-4467-823f-613825dce729'), 9: UUID('8

### Data preview

In [10]:
# Show data preview
jeopardy = client.collections.use("Jeopardy")
response=jeopardy.query.fetch_objects(limit=4)

# TODO: fetch 4 objects
# response = jeopardy

for item in response.objects:
    print(item.uuid, item.properties)

343cb976-b2e9-4e6f-add3-7a1c326ffa8d {'answer': 'DNA', 'question': 'In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance', 'category': 'SCIENCE'}
694ced5b-f355-449d-a866-0701561621d5 {'answer': 'Liver', 'question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'category': 'SCIENCE'}
74f3e196-f573-4759-80e4-a5ae0b712e51 {'answer': 'wire', 'question': 'A metal that is ductile can be pulled into this while cold & under pressure', 'category': 'SCIENCE'}
7aa8a88a-92e1-4a3c-9a3b-660fbfd3a507 {'answer': 'the diamondback rattler', 'question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'category': 'ANIMALS'}


In [11]:
# Show data preview - with vectors
response = jeopardy.query.fetch_objects(
    limit=4,
    include_vector=True
    # TODO: add include_vectors
)

for item in response.objects:
    print(item.properties)
    print(item.vector, '\n')

{'answer': 'DNA', 'question': 'In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance', 'category': 'SCIENCE'}
{'default': [0.010430101305246353, 0.04145771265029907, -0.014448109082877636, 0.06629413366317749, -0.05726705119013786, 0.003874720074236393, 0.022460242733359337, -0.03627549856901169, -0.04069351777434349, -0.012836129404604435, 0.058747682720422745, -0.023487132042646408, 0.017421314492821693, -0.022293074056506157, -0.0474996492266655, -0.005113556049764156, 0.03601280599832535, 0.02753499150276184, -0.008859914727509022, 0.009427092969417572, 0.005480729043483734, 0.004047858528792858, 0.01385108008980751, 0.011982377618551254, 0.059368591755628586, -0.03997708112001419, -0.023570716381072998, -0.023140855133533478, 0.04253236576914787, -0.017994463443756104, 0.016549652442336082, -0.04055022820830345, 0.044299572706222534, 0.01355256512761116, -0.020955728366971016, 0.03770837187767029, 0.03940393403172493, -0.0125495558604

### Super quick query example

In [12]:
# TODO: add near text query, search for African animals with limit 2
# response = jeopardy.query
response = jeopardy.query.near_text(
    query= "african animals",
    limit=2
)

for item in response.objects:
    print(item.properties)

{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}


## A bit bigger example - 2k objects

### Load data

In [13]:
import json

with open("./wiki-2k.json") as file:
    data_2k = json.load(file)

print(json.dumps(data_2k[0:2], indent=2))

[
  {
    "text": "At this point in the siege, Lee's army had strengthened the Petersburg line. They dug breastworks out of rifle pits. At night, with pick and shovel, they then turned the breastworks into  deep trenches. Pointed stakes turned outwards were designed to break up any frontal attacks. The area between the two lines became a no man's land. The summer that year was hot and dry. Streams and springs were quickly drying up causing a water shortage on both sides. The siege was quickly becoming a stalemate.",
    "title": "Siege of Petersburg",
    "url": "https://simple.wikipedia.org/wiki/Siege%20of%20Petersburg",
    "wiki_id": "20231101.simple_550339_9"
  },
  {
    "text": "1944  Holocaust: Anne Frank and her family are placed on the last transport train from the Westerbork transit camp to Auschwitz.",
    "title": "September 3",
    "url": "https://simple.wikipedia.org/wiki/September%203",
    "wiki_id": "20231101.simple_8532_17"
  }
]


### Create a collection with Named Vectors and SourceProperties

In [14]:
from weaviate.classes.config import Configure, Property, DataType

def create_wiki_collection():
    if client.collections.exists("Wiki"):
        client.collections.delete("Wiki")

    # Create a collection here - with Weaviate vectorizer and define source properties
    client.collections.create(
        name="Wiki",

        vector_config=[
            # NOTE: we are using NamedVectors here
            Configure.Vectors.text2vec_openai(
                name="main_vector",
                model="text-embedding-3-small",
                base_url=OPENAI_URL,
                source_properties=['title', 'text']

                # TODO: set source properties to "title" and "text"
                # source_properties=[] # which properties should be used to generate a vector
            )
        ],

        # Example: how to define property schema (Optional)
        properties=[  
            Property(name="title", data_type=DataType.TEXT),
            Property(name="text", data_type=DataType.TEXT),
            Property(name="url", data_type=DataType.TEXT),
            Property(name="wiki_id", data_type=DataType.TEXT),
        ],
    )

create_wiki_collection()

### Import data - 2k objects with Batch

Batch speeds up the import process by grouping objects to be added in bigger batch groups.

Batch creates an internal buffer to collect objects to be added.<br>
Each time the buffer count reaches `batch_size`, batch sends the new objects to Weaviate.

Types of batch:
* `dynamic` - let batch calculate the optimal batch_size based on detected latency
* `fixed_size` - provide a fixed batch_size
* `rate_limit` - limit the number of requests (per minute), useful for working with models with a rate limit

### Take 1 – import sample 100

In [19]:
from tqdm import tqdm

sample_100 = data_2k[0:100]

wiki = client.collections.use("Wiki")

with wiki.batch.dynamic() as batch:
    for item in tqdm(sample_100, total=len(sample_100), desc="Uploading"):
        batch.add_object(item)

# TODO: setup dynamic batch
# loop through the sample_100 data
# add each object to the batch

print(f"Wiki count: {len(wiki)}")

Uploading: 100%|██████████| 100/100 [00:00<00:00, 26645.73it/s]


Wiki count: 400


In [16]:
# check for errors
if(len(wiki.batch.failed_objects)>0):
    print("Import complete with errors")
    for err in wiki.batch.failed_objects:
        print(err)
else:
    print("Import complete with no errors")

Import complete with no errors


### Take 2 – import sample 100 – with UUID

To avoid inserting duplicates, you can generate a UUID based on the whole object or a unique property.

In [20]:
from weaviate.util import generate_uuid5

print(generate_uuid5("This UUID is always the same"))
print(generate_uuid5("This UUID is always the same"))
print(generate_uuid5("This UUID is always the same"))
print("====================================")

print(generate_uuid5("This UUID is different"))
print(generate_uuid5("This UUID is different"))
print("====================================")

obj1 = { "title": "this is an object", "count": 1 }
obj2 = { "title": "this is an object", "count": 2 }
print(generate_uuid5(obj1))
print(generate_uuid5(obj2))


8d3441c0-c1d1-5859-8a5e-efce9e7d3bd8
8d3441c0-c1d1-5859-8a5e-efce9e7d3bd8
8d3441c0-c1d1-5859-8a5e-efce9e7d3bd8
09f975a6-0e62-565a-982e-e6ce148eac86
09f975a6-0e62-565a-982e-e6ce148eac86
c3c3ad32-fa65-5944-a021-415f8fda02af
4d0b77d3-4862-59bc-bf9f-9fe2b9bf89f0


In [21]:
# recreate the collection to start again
create_wiki_collection()

> Rerun the import script multiple times.

> Starting from the second run, the script should finish a lot faster, and the wiki count shouldn't increase.

In [27]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

sample_100 = data_2k[0:100]

wiki = client.collections.get("Wiki")

with wiki.batch.fixed_size(batch_size=50, concurrent_requests=2) as batch:
    for item in tqdm(sample_100, total=len(sample_100), desc="Uploading"):
        # TODO: generate an id from item["wiki_id"]
        id = generate_uuid5(item["wiki_id"])

        batch.add_object(
            item,
            uuid=id

            # TODO: provide the new id here 
            # uuid=
        )

print(f"Wiki count: {len(wiki)}")

Uploading: 100%|██████████| 100/100 [00:00<00:00, 6690.87it/s]

Wiki count: 100


### Take 2 - import the rest of the data - but break if multiple errors

In [28]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

wiki = client.collections.get("Wiki")

with wiki.batch.fixed_size(batch_size=100, concurrent_requests=2) as batch:
    for item in tqdm(data_2k, total=len(data_2k), desc="uploading"):
        id = generate_uuid5(item["wiki_id"])
        batch.add_object(item, uuid=id)

        # Check number of errors while running
        if(batch.number_errors > 10):
            print("Errors during batch import")
            break

uploading: 100%|██████████| 2000/2000 [00:09<00:00, 215.98it/s]


### Check for errors

In [29]:
if(len(wiki.batch.failed_objects)>0):
    print("Import complete with errors")
    for err in wiki.batch.failed_objects:
        print(err)
else:
    print("Import complete with no errors")

Import complete with no errors


## Bonus - iterate through all collection data

The client has a built-in function that allows you to iterate through all collection data.

In [30]:
wiki = client.collections.get("Wiki")

counter = 100

for item in wiki.iterator():
    print(item.properties)

    if (counter == 0): break
    
    counter -= 1

{'text': 'Mission Hills is an urban residential community of the San Fernando Valley, within the city of Los Angeles, California.', 'title': 'Mission Hills, Los Angeles', 'wiki_id': '20231101.simple_512123_0', 'url': 'https://simple.wikipedia.org/wiki/Mission%20Hills%2C%20Los%20Angeles'}
{'text': 'Pavlov researched classical conditioning through the use of dogs and their natural ability to salivate, produce water in their mouths. Thorndike and Watson rejected looking at one\'s own conscious thoughts and feelings ("Introspection"). They wanted to restrict psychology to experimental methods. Skinner\'s research leant mainly on behavior shaping using positive reinforcement (rewards rather than punishments).', 'title': 'Behaviorism', 'wiki_id': '20231101.simple_260848_4', 'url': 'https://simple.wikipedia.org/wiki/Behaviorism'}
{'text': 'On 27 April 1970 he was selected as a cosmonaut and flew as Commander on Soyuz T-5. He retired on 31 October 1992 due to medical reasons after injuries he 

You can also get `vector embeddings`, by using `include_vector`.

In [31]:
counter = 10

for item in wiki.iterator(include_vector=True):
    print(item.properties)
    print(item.vector)

    if (counter == 0): break
    
    counter -= 1

{'text': 'Mission Hills is an urban residential community of the San Fernando Valley, within the city of Los Angeles, California.', 'title': 'Mission Hills, Los Angeles', 'wiki_id': '20231101.simple_512123_0', 'url': 'https://simple.wikipedia.org/wiki/Mission%20Hills%2C%20Los%20Angeles'}
{'main_vector': [-0.03407542407512665, -0.0327276811003685, 0.01582474447786808, 0.010175456292927265, -0.031177779659628868, -0.026056356728076935, -0.01400529220700264, 0.003571517998352647, 0.008221229538321495, -0.00500911008566618, -0.008951256982982159, 0.01686924509704113, 0.0010810018284246325, 0.010995333082973957, -0.035378243774175644, -0.009310655295848846, -0.006985798943787813, 0.021676193922758102, -0.003021189710125327, 0.02248483896255493, 0.0008030299795791507, -0.009378042072057724, -0.03722015768289566, 0.04034242779016495, 0.017509423196315765, -0.0390845350921154, -0.04027504101395607, 0.021137095987796783, -0.0016481769271194935, -0.010343924164772034, -0.010147378779947758, -0.0

## Close the client

In [32]:
client.close()